In [ ]:
#import libraries, dependencies
import pandas as pd
import numpy as np
import requests
import json
from config import api_key

#Import CSV and create a data frame
#Using a CSV file with all zip codes in Denver from:
#https://www.downloadexcelfiles.com/us_en/download-list-us-zip-codes-colorado-state#gsc.tab=0
#we selected only zip codes in Denver City using excel and saved  as denver_zipcodes.csv
file = 'resources/denver_zipcodes.csv'
denver = file_one_df = pd.read_csv(file)
denver_df = pd.DataFrame(denver)
#adding columns for Population, Per Capita Income, Median Age
denver_df = denver_df.assign(Population = "")
denver_df = denver_df.assign(Income = "")
denver_df.rename(columns= {'Income': 'Per Capita Income'}, inplace=True)
denver_df =denver_df.assign(Age = "")
denver_df.rename(columns= {'Age': 'Median Age'}, inplace=True)

#TODO: select parameters for request to geoapify, which will find all healthcare facilities in each zip code(Kelsy)

#TODO: Use a for loop to make an api request for a radius around Denver, and place the healthcare facility results into a DF(Kelsy).
#include each facility's name, zip code, subcategory(type), and wheelchair access status

#Get population, median age, per capita income, of each zip code and add to data frame with healthcare facility info (Lisa)
#from https://data.colorado.gov/ (Lisa)
zip_codes = denver_df['Zip Code'].values.tolist()
zip_codes 

url_pop = 'https://data.colorado.gov/resource/ucnv-vw74.json?'
for index, row in denver_df.iterrows():
    #create the parameters, full url for the API call
    params_pop = {'zip_code': denver_df.loc[index, "Zip Code"],
              }
    response_pop = requests.get(url_pop, params=params_pop)
    response_pop = response_pop.json()

    # Store the zip's population, per capita income and median age in the denver_df DataFrame
    try:
        denver_df.loc[index, "Population"] = response_pop[0]["pop"]
        
    except (KeyError, IndexError):
        denver_df.loc[index, "Population"] = np.nan
    try:
        denver_df.loc[index, "Per Capita Income"] = response_pop[0]['per_cap_in']
    except (KeyError, IndexError):
        denver_df.loc[index, "Per Capita Income"] = np.nan
    try: 
        denver_df.loc[index, 'Median Age'] = response_pop[0]['med_age']
    except (KeyError, IndexError):
        denver_df.loc[index, "Median Age"] = np.nan
denver_df.dropna()

#Question 1: are healthcare facilities more dense where the population is dense? In other words: 
#Is there a correlation between pop of zip code and  number of facilities in the zip code?
#TODO: plot #of healthcare facilities per zip code against population per zip code (double bars with zip as x axis)
#TODO: scatter plot and linear regression of facility count vs population

#Question 2: Is there a correlation of per capita income with # of healthcare facilities by zip code in Denver?
#TODO: plot #of healthcare facilities per zip code against per cap income
#TODO: scatter plot and lin reg of  zip code against per cap income 

#Question 3: Is there a correlation of median age with # of healthcare facilities by zip code in Denver?
#TODO: plot #of healthcare facilities per zip code against median age
#TODO: scatter plot and lin reg of  zip code against median age

#Quesion 4: What percentage of facilities are wheelchair accessible and what percentage by subcategory?
#TODO: count # and find % of facilities with wheelchair access
#TODO: If not very close to 100%, plot types of facility against wheelchair access(bar)
#TODO: map with color of facility dot by wheelchair access

#TODO: written analysis

#TODO: prepare slides for presentation

#TODO: practice presentation(or at least be clear on respective roles)


In [74]:
#example call to geoapify(delete this cell when looped calls are complete)
url = 'https://api.geoapify.com/v2/places'
latitude = -33.8698439
longitude = 151.2082848
radius = 5000
filters = f'circle: {longitude}, {latitude}', {radius}
bias = f"proximity:{longitude},{latitude}"
limit = 5
parameters = {
    'categories': "healthcare",
    "limit": limit,
    "filters":filters,
    "bias": bias,
    "apiKey":api_key}

results = requests.get(url , params=parameters)    
results.json()       


{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}

In [75]:
print(results.url)

https://api.geoapify.com/v2/places?categories=healthcare&limit=5&filters=circle%3A+151.2082848%2C+-33.8698439&filters=5000&bias=proximity%3A151.2082848%2C-33.8698439&apiKey=your+key+here
